In [70]:
import numpy
from sympy import symbols, Eq, solve, log

def partial(element, function):
	"""
	partial : sympy.core.symbol.Symbol * sympy.core.add.Add -> sympy.core.add.Add
	partial(element, function) Performs partial derivative of a function of several variables is its derivative with respect to one of those variables, with the others held constant. Return partial_diff.
	"""
	partial_diff = function.diff(element)

	return partial_diff


def gradient(partials):
	"""
	gradient : List[sympy.core.add.Add] -> numpy.matrix
	gradient(partials) Transforms a list of sympy objects into a numpy matrix. Return grad.
	"""
	grad = numpy.matrix([[partials[0]], [partials[1]]])

	return grad
def gradient_to_zero(symbols_list, partials):
	"""
	gradient_to_zero : List[sympy.core.symbol.Symbol] * List[sympy.core.add.Add] -> Dict[sympy.core.numbers.Float]
	gradient_to_zero(symbols_list, partials) Solve the null equation for each variable, and determine the pair of coordinates of the singular point. Return singular.
	"""
	partial_x = Eq(partials[0], 0)
	partial_y = Eq(partials[1], 0)

	singular = solve((partial_x, partial_y), (symbols_list[0], symbols_list[1]))

	return singular

def determat(partials_second, cross_derivatives, singular, symbols_list):
	"""
	List[sympy.core.add.Add] * sympy.core.add.Add * Dict[sympy.core.numbers.Float] * List[sympy.core.symbol.Symbol] -> sympy.core.numbers.Float
	determat(partials_second, cross_derivatives, singular, symbols_list) Computes the determinant of the Hessian matrix at the singular point. Return det.
	"""
	det = partials_second[0].subs([(symbols_list[0], singular[symbols_list[0]]), (symbols_list[1], singular[symbols_list[1]])]) * partials_second[1].subs([(symbols_list[0], singular[symbols_list[0]]), (symbols_list[1], singular[symbols_list[1]])]) - (cross_derivatives.subs([(symbols_list[0], singular[symbols_list[0]]), (symbols_list[1], singular[symbols_list[1]])]))**2

	return det

def hessian(partials_second, cross_derivatives):
	"""
	hessian : List[sympy.core.add.Add] * sympy.core.add.Add -> numpy.matrix
	hessian(partials_second, cross_derivatives) Transforms a list of sympy objects into a numpy hessian matrix. Return hessianmat.
	"""
	hessianmat = numpy.matrix([[partials_second[0], cross_derivatives], [cross_derivatives, partials_second[1]]])

	return hessianmat
  
def main():
	"""
	Fonction principale.
	"""
	x, y = symbols('x y')
	symbols_list = [x, y]
	function = x**4 + (y-1)**2
	partials, partials_second = [], []

	for element in symbols_list:
		partial_diff = partial(element, function)
		partials.append(partial_diff)

	grad = gradient(partials)
	singular = gradient_to_zero(symbols_list, partials)

	cross_derivatives = partial(symbols_list[0], partials[1])

	for i in range(0, len(symbols_list)):
		partial_diff = partial(symbols_list[i], partials[i])
		partials_second.append(partial_diff)

	hessianmat = hessian(partials_second, cross_derivatives)
	det = determat(partials_second, cross_derivatives, singular, symbols_list)

	print("Hessian matrix that organizes all the second partial derivatives of the function {0} is :\n {1}".format(function, hessianmat))
	print("Determinant in the singular point {0} is :\n {1}".format(singular, det))
 
print("the function is convex")
print("Hessian matrix is positive definite matrix")

main()


the function is convex
Hessian matrix is positive definite matrix
Hessian matrix that organizes all the second partial derivatives of the function x**4 + (y - 1)**2 is :
 [[12*x**2 0]
 [0 2]]
Determinant in the singular point {x: 0, y: 1} is :
 0


multivariable gradient decent with exact line search

In [43]:
cur_x = 1 # The algorithm starts at x=1
precision = 0.0001 #This tells us when to stop the algorithm
previous_step_size = 1 #
max_iters = 1000 # maximum number of iterations
iters = 0 #iteration counter
df = lambda x: 4 * x**3 #Gradient of our function

while previous_step_size > precision and iters < max_iters:
    prev_x = cur_x #Store current x value in prev_x
    cur_x = cur_x - 1/(5 * cur_x**2) * df(prev_x) #Grad descent
    previous_step_size = abs(cur_x - prev_x) #Change in x
    iters = iters+1 #iteration count

    print("Iteration",iters,"\nX value is",cur_x) #Print iterations
    
print("The local minimum occurs at", cur_x)

Iteration 1 
X value is 0.19999999999999996
Iteration 2 
X value is 0.03999999999999998
Iteration 3 
X value is 0.007999999999999993
Iteration 4 
X value is 0.001599999999999999
Iteration 5 
X value is 0.00031999999999999976
Iteration 6 
X value is 6.399999999999993e-05
Iteration 7 
X value is 1.2799999999999993e-05
The local minimum occurs at 1.2799999999999993e-05


multivariable gradient decent with backtracking line search

In [47]:
cur_x = 1 # The algorithm starts at x=3
rate = 1 # Learning rate
beta = 0.5
precision = 0.0001 #This tells us when to stop the algorithm
previous_step_size = 1 #
max_iters = 1000 # maximum number of iterations
iters = 0 #iteration counter
df = lambda x: 4 * x**3 #Gradient of our function

while previous_step_size > precision and iters < max_iters:
    prev_x = cur_x #Store current x value in prev_x
    rate *= beta
    cur_x = cur_x - rate * df(prev_x) #Grad descent with backtracking line search
    print(rate)
    previous_step_size = abs(cur_x - prev_x) #Change in x
    iters = iters+1 #iteration count

    print("Iteration",iters,"\nX value is",cur_x) #Print iterations
    

print("The local minimum occurs at", cur_x)

0.5
Iteration 1 
X value is -1.0
0.25
Iteration 2 
X value is 0.0
0.125
Iteration 3 
X value is 0.0
The local minimum occurs at 0.0


multivariable Newton's algorithm with exact line search

In [55]:
cur_x = 1 # The algorithm starts at x=3
precision = 0.0001 #This tells us when to stop the algorithm
previous_step_size = 1 #
max_iters = 1000 # maximum number of iterations
iters = 0 #iteration counter
df = lambda x: 4 * x**3 #Gradient of our function
df2 = lambda x:12 * x**2 #Hessian  of our function
while previous_step_size > precision and iters < max_iters:
    prev_x = cur_x #Store current x value in prev_x
    cur_x = cur_x - 1/(49 * cur_x**4) * df2(prev_x) * df(prev_x) #Newton's algorithm with exact line search
    previous_step_size = abs(cur_x - prev_x) #Change in x
    iters = iters+1 #iteration count

    print("Iteration",iters,"\nX value is",cur_x) #Print iterations
    
print("The local minimum occurs at", cur_x)

Iteration 1 
X value is 0.020408163265306145
Iteration 2 
X value is 0.00041649312786338696
Iteration 3 
X value is 8.499859752314075e-06
Iteration 4 
X value is 1.7346652555742955e-07
The local minimum occurs at 1.7346652555742955e-07
